In [1]:
# Importa as bibliotecas necessárias
import wget
import pandas as pd
import json
import datetime
import os
from pandas import json_normalize

In [2]:
PASTA_TEMPORARIOS = '../../dados/temporarios/'
PASTA_AUXILIARES = '../../dados/auxiliares/'
PASTA_SAIDA = '../../dados/'

In [3]:
# Define os dados para download
url = 'http://plataforma.saude.gov.br/novocoronavirus/resources/scripts/database.js'
arquivo = os.path.join(PASTA_TEMPORARIOS,'database.js')

In [4]:
# Baixa o arquivo JS
wget.download(url, arquivo)

'../../dados/temporarios/database.js'

In [5]:
# Converte o arquivo JavaScript para JSON
base_js = open(arquivo, "rt")
base_json = open(
    os.path.join(PASTA_TEMPORARIOS,"database.json"),
    "wt"
)

for line in base_js:
	base_json.write(line.replace('var database=', ''))

base_js.close()
base_json.close()

In [7]:
# Exporta o dataframe
df = json.load(open(os.path.join(PASTA_TEMPORARIOS,"database.json")))
df = json_normalize(data=df['brazil'], record_path='values',meta=['date','time'])

In [8]:
# Carrega a tabela com IDs e estados
indice = pd.read_csv(os.path.join(
    PASTA_AUXILIARES,
    'indice.csv'
))

# Transforma as IDs em string
df.uid = df.uid.astype(str)
indice.uid = indice.uid.astype(str)

# Insere as UFs nos dados
df = pd.merge(df, indice, on='uid', how='left')

In [9]:
# Transforma números em inteiros
dados_numericos = ['suspects', 'refuses', 'confirmado', 'deads','cases']

for coluna in dados_numericos:
    df[coluna] = df[coluna].fillna(0)
    df[coluna] = df[coluna].astype(int)

In [11]:
# Exporta a base em CSV
dados = os.path.join(PASTA_SAIDA, 'corona_brasil' + '.csv')
df.to_csv(dados, index = False)